# MongoDB & Python

- **MongoDB** is a document store database that works with collections containing multiple documents. MongoDB has a master-slave architecture
- **Cassandra** is a column-oriented database. Cassandra has a peer-to-peer architecture where all are master nodes in communication with each other.

In [12]:
import pymongo
from pymongo import MongoClient
client = MongoClient(
    host="localhost", #establishes connection to the host (localhost) and port (27017), from docker
    username= "admin",
    password= "password",
    port=27017)

print(client)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [16]:
for db in client.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 102400, 'empty': False}
{'name': 'config', 'sizeOnDisk': 110592, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728, 'empty': False}


## Collection
A collection is a grouping of MongoDB documents. Documents within a collection can have different fields. A collection is the equivalent of a table in a relational database system. 

In [22]:
# Connect to a particular db or if this db does not exist it will create it.
my_db = client["mydatabase"] # Important ---> In MongoDB, a database is not created until it gets content!

# To create a collection in MongoDB, use database object and specify the name of the collection you want to create.
my_col = mydb["onecollection"] # Important ---> In MongoDB, a collection is not created until it gets content!

### Insert

In [22]:
#To insert a record, or document as it is called in MongoDB, into a collection, we use the insert_one() method.
my_record_dict_one = { "name": "John", "address": "Highway 37" }
insert_result = my_col.insert_one(my_record_dict_one)
print(insert_result.inserted_id)

637e68fbcabf95f7f04c0b96


In [23]:
for db in client.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 102400, 'empty': False}
{'name': 'config', 'sizeOnDisk': 110592, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728, 'empty': False}
{'name': 'mydatabase', 'sizeOnDisk': 40960, 'empty': False}


In [47]:
my_db = client["mydatabase"]
for collection in my_db.list_collections():
    print(collection)
    key_list = list(collection.keys())
    values_list = list(collection.values())
    print("keys:",key_list,"---> values:",values_list)
    print(collection['name'])
    print(type(collection))

{'name': 'onecollection', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'M\xc1\x9b\x0eg\xf7G\xc3\x87\xa0\x9e\x8bs"\x8c\x7f', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
keys: ['name', 'type', 'options', 'info', 'idIndex'] ---> values: ['onecollection', 'collection', {}, {'readOnly': False, 'uuid': Binary(b'M\xc1\x9b\x0eg\xf7G\xc3\x87\xa0\x9e\x8bs"\x8c\x7f', 4)}, {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}]
onecollection
<class 'dict'>


In [ ]:
    client = MongoClient("localhost", 27017, maxPoolSize=50)
    db = client.localhost
    collection = db['chain']
    cursor = collection.find({})

In [51]:
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
my_cursor = my_onecollection.find({})
for document in my_cursor:
    print(document)

{'_id': ObjectId('637e68cbcabf95f7f04c0b95'), 'name': 'John', 'address': 'Highway 37'}
{'_id': ObjectId('637e68fbcabf95f7f04c0b96'), 'name': 'John', 'address': 'Highway 37'}


### Return Only Some Fields 
- Return only the names and the addresses and not the _ids

In [58]:
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
my_cursor = my_onecollection.find({},{ "_id": 0, "name": 1, "address": 1 }) # 0 not return, 1 return
# Important:  If you specify a field with the value 0, all other fields get the value 1, and vice versa:
for document in my_cursor:
    print(document)

{'name': 'John', 'address': 'Highway 37'}
{'name': 'John', 'address': 'Highway 37'}


### Filter the Result
- you can filter the result by using a query object

In [60]:
my_record_dict_one = { "name": "John", "address": "Park Lane 38" }
insert_result = my_col.insert_one(my_record_dict_one)
print(insert_result.inserted_id)

637e7a37cabf95f7f04c0b97


In [61]:
my_cursor = my_onecollection.find({})
for document in my_cursor:
    print(document)

{'_id': ObjectId('637e68cbcabf95f7f04c0b95'), 'name': 'John', 'address': 'Highway 37'}
{'_id': ObjectId('637e68fbcabf95f7f04c0b96'), 'name': 'John', 'address': 'Highway 37'}
{'_id': ObjectId('637e7a37cabf95f7f04c0b97'), 'name': 'John', 'address': 'Park Lane 38'}


In [62]:
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
myquery = { "address": "Park Lane 38" }
my_cursor = my_onecollection.find(myquery) # 0 not return, 1 return
# Important:  If you specify a field with the value 0, all other fields get the value 1, and vice versa:
for document in my_cursor:
    print(document)

{'_id': ObjectId('637e7a37cabf95f7f04c0b97'), 'name': 'John', 'address': 'Park Lane 38'}


### Advanced Query
- Find documents where the address starts with the letter "P" or higher:

In [65]:
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
myquery = { "address": { "$gt": "P" } }
my_cursor = my_onecollection.find(myquery) # 0 not return, 1 return
# Important:  If you specify a field with the value 0, all other fields get the value 1, and vice versa:
for document in my_cursor:
    print(document)

{'_id': ObjectId('637e7a37cabf95f7f04c0b97'), 'name': 'John', 'address': 'Park Lane 38'}


- Find documents where the address starts with the letters "ar":

In [66]:
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
myquery = { "address": { "$regex": "ar" } }
my_cursor = my_onecollection.find(myquery) # 0 not return, 1 return
# Important:  If you specify a field with the value 0, all other fields get the value 1, and vice versa:
for document in my_cursor:
    print(document)

{'_id': ObjectId('637e7a37cabf95f7f04c0b97'), 'name': 'John', 'address': 'Park Lane 38'}


### Delete Document
- delete_one()

In [67]:
print("Before Delete one")
my_cursor = my_onecollection.find({})
for document in my_cursor:
    print(document)

print("After Delete one ")
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
myquery = { "address": "Highway 37" }
my_onecollection.delete_one(myquery)

Before Delete one
{'_id': ObjectId('637e68cbcabf95f7f04c0b95'), 'name': 'John', 'address': 'Highway 37'}
{'_id': ObjectId('637e68fbcabf95f7f04c0b96'), 'name': 'John', 'address': 'Highway 37'}
{'_id': ObjectId('637e7a37cabf95f7f04c0b97'), 'name': 'John', 'address': 'Park Lane 38'}
After Delete one 


In [68]:
my_cursor = my_onecollection.find({})
for document in my_cursor:
    print(document)

{'_id': ObjectId('637e68fbcabf95f7f04c0b96'), 'name': 'John', 'address': 'Highway 37'}
{'_id': ObjectId('637e7a37cabf95f7f04c0b97'), 'name': 'John', 'address': 'Park Lane 38'}


- delete_many

In [ ]:
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
myquery = { "address": "Highway 37" }
x = my_onecollection.delete_many(myquery)
print(x.deleted_count, " documents deleted.")

- To delete all documents in a collection, pass an empty query object to the delete_many() method

### Delete Collection

In [ ]:
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
my_onecollection.drop()

### Update Collection

In [ ]:
my_db = client["mydatabase"]
my_onecollection = my_db['onecollection']
myquery = { "address": "Valley 345" }
newvalues = { "$set": { "address": "Canyon 123" } }
my_onecollection.update_one(myquery, newvalues)

# Reference

- Python+Mongo: https://www.w3schools.com/python/python_mongodb_create_db.asp
- Cloud+Python+Mongo tutorial: https://www.youtube.com/watch?v=qWYx5neOh2s
- Collections: https://www.mongodb.com/docs/compass/current/collections
- Mongo vs Cassandra: https://www.knowledgehut.com/blog/data-science/cassandra-vs-mongodb
